In [ ]:
!pip install -q langchain-community langchain-groq langchain shutup langchain-text-splitters langchain-core sentence-transformers faiss-cpu faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.3 MB/s eta 0:00:00


In [ ]:
import os
import shutup
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

shutup.please()

In [ ]:
file_path = "/content/Conditional Generative Adversarial Nets.txt"
loader = TextLoader(file_path)
docs = loader.load()
docs

[Document(page_content='arXiv:1411.1784v1 [cs.LG] 6 Nov 2014\n\nConditional Generative Adversarial Nets\nMehdi Mirza\nDépartement d’informatique et de recherche opérationnelle\nUniversité de Montréal\nMontréal, QC H3C 3J7\nmirzamom@iro.umontreal.ca\nSimon Osindero\nFlickr / Yahoo Inc.\nSan Francisco, CA 94103\nosindero@yahoo-inc.com\n\nAbstract\n\nGenerative Adversarial Nets [8] were recently introduced as a novel way to train\ngenerative models. In this work we introduce the conditional version of generative\nadversarial nets, which can be constructed by simply feeding the data, y, we wish\nto condition on to both the generator and discriminator. We show that this model\ncan generate MNIST digits conditioned on class labels. We also illustrate how\nthis model could be used to learn a multi-modal model, and provide preliminary\nexamples of an application to image tagging in which we demonstrate how this\napproach can generate descriptive tags which are not part of training labels.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
documents

[Document(page_content='arXiv:1411.1784v1 [cs.LG] 6 Nov 2014\n\nConditional Generative Adversarial Nets\nMehdi Mirza\nDépartement d’informatique et de recherche opérationnelle\nUniversité de Montréal\nMontréal, QC H3C 3J7\nmirzamom@iro.umontreal.ca\nSimon Osindero\nFlickr / Yahoo Inc.\nSan Francisco, CA 94103\nosindero@yahoo-inc.com\n\nAbstract\n\nGenerative Adversarial Nets [8] were recently introduced as a novel way to train\ngenerative models. In this work we introduce the conditional version of generative\nadversarial nets, which can be constructed by simply feeding the data, y, we wish\nto condition on to both the generator and discriminator. We show that this model\ncan generate MNIST digits conditioned on class labels. We also illustrate how\nthis model could be used to learn a multi-modal model, and provide preliminary\nexamples of an application to image tagging in which we demonstrate how this\napproach can generate descriptive tags which are not part of training labels.

In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [ ]:
groq_api_key = GROQ_API_KEY

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
# HF_TOKEN

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name = "mixedbread-ai/mxbai-embed-large-v1", # mixedbread-ai/mxbai-embed-large-v1
                                  encode_kwargs = {"precision" : "binary"},
                                  )
vector = FAISS.from_documents(documents, embeddings)
retriver = vector.as_retriever()
model = ChatGroq(temperature = 0.0,
                 groq_api_key = groq_api_key,
                 model_name = "mixtral-8x7b-32768") # mixtral-8x7b-32768
prompt = ChatPromptTemplate.from_template("""Answer the following questionbased only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(model, prompt)
retrival_chain = create_retrieval_chain(retriver, document_chain)

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
%%time
qurey = "Give me the discriminator loss."
response = retrival_chain.invoke({"input": str(qurey)})
print(response["answer"])

The discriminator loss is the second term of the objective function in Eq 2, which is the negative log-likelihood of the true label for real data and the negative log-likelihood of the generated label for fake data. It is expressed as:

Ez∼pz(z) [log(1 − D(G(z|y)))]

where z is the noise prior, y is the conditioning input, G is the generator function, and D is the discriminator function.
CPU times: user 363 ms, sys: 4.77 ms, total: 368 ms
Wall time: 1.92 s


In [ ]:
from IPython.display import Markdown
Markdown(response["answer"])

The discriminator loss is the second term of the objective function in Eq 2, which is the negative log-likelihood of the true label for real data and the negative log-likelihood of the generated label for fake data. It is expressed as:

Ez∼pz(z) [log(1 − D(G(z|y)))]

where z is the noise prior, y is the conditioning input, G is the generator function, and D is the discriminator function.